In [ ]:
# Mout Google Drive
# https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
# %pwd %ls
# run github settings
%run /content/drive/MyDrive/CNNStanford/pytorch/pytorch_sandbox/Colab_Helper.ipynb

In [ ]:
MESSAGE = "clean file & gitignore again"
!git config --global user.email "ronyginosar@mail.huji.ac.il"
!git config --global user.name "ronyginosar"
!git add .

In [ ]:
!git commit -m "{MESSAGE}"
!git push "{GIT_PATH}"

Empty

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
from PIL import Image
import torchvision.transforms as transforms

dataset_root = ''# add path cityscapes

# 1. Implement the custom Dataset
class Cityscapes(Dataset):
    def __init__(self, dataset_root, split, ...):
        assert split in ['train','val','test']
        ...

    def __len__(self):
        return ...

    def __getitem__(self, idx):
        ...
        return img, seg

    # visualize a sample of the dataset (index idx)
    def show(self, idx):
        ...

# 2. Create an instance of the dataset and verify by visualization
dataset = Cityscapes(...)

# 3. Add data augmentation (to both input & GT) and verify
# Resize to (256,512)  -> Random Crop (256,256) -> Random Horizontal Flip
# hint – what type of interpolation to use for resizing?
# Data & GT augmentations must be matched, so write custom ‘pairwise’ transforms
# Hint – For Resize, use an existing functional transform: transforms.functional.resize
# Hint – For random crop, use transforms.RandomCrop.get_params to get the crop indices
# Use transforms.Compose on the custom augmentations

# For example:
class RandomHorizontalFlip_pairwise:
    def __init__(self, p=0.5):
        super().__init__()
        ...

    def __call__(self, img, mask):
        ...
        transformed_img = ...
        transformed_mask = ...
        return transformed_img, transformed_mask


# 4. Create a dataloader, generate batches and check validity (+ visualize!)






Solution

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
from PIL import Image
import torch
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

root_dir = ''# add path cityscapes

class Cityscapes(Dataset):
    def __init__(self, root_dir, split, pairwise_transform=None):
        assert split in ['train','val','test']
        self.root_dir = root_dir
        self.split = split
        self.images = sorted(list((Path(root_dir)/"leftImg8bit"/split).rglob("*.png")))
        self.labelIds =[Path(root_dir)/"gtFine"/split/path.parent.name/
                        path.name.replace("_leftImg8bit.png",f"_gtFine_labelIds.png")
                        for path in self.images]
        self.pairwise_transform = pairwise_transform

    def __len__(self):
        return len(self.images)

    @staticmethod
    def read_seg_file(file):
        segm = Image.open(file)
        segm = torch.from_numpy(np.array(segm, np.uint8))
        segm = segm.unsqueeze(0)
        return segm

    def __getitem__(self, idx):
        # Get the image
        img = transforms.ToTensor()(Image.open(self.images[idx]))
        # Get the GT
        segm = self.read_seg_file(self.labelIds[idx])
        # Apply pairwise transform
        if self.pairwise_transform is not None:
            img, segm = self.pairwise_transform((img, segm))

        return img, segm

    def show(self, idx):
        img, segm = self[idx]
        fig, (ax_im, ax_seg) = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(16,4))
        ax_im.imshow(img.permute(1,2,0).cpu().numpy())
        ax_im.set(title="image")
        ax_seg.imshow(segm.permute(1,2,0).cpu().numpy())
        ax_seg.set(title="GT segmentation")

class PairwiseRandomHorizontalFlip:
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, sample):
        img, target = sample
        if torch.rand(1) < self.p:
            return transforms.functional.hflip(img), transforms.functional.hflip(target)
        return sample

class PairwiseResize:
    def __init__(self, output_size,
                     interp_img=transforms.InterpolationMode.BILINEAR,
                     interp_tgt=transforms.InterpolationMode.NEAREST):
        self.resize_image = transforms.Resize(output_size, interpolation=interp_img)
        self.resize_target = transforms.Resize(output_size, interpolation=interp_tgt)

    def __call__(self, sample):
        assert isinstance(sample, tuple) and len(sample)==2
        assert sample[0].shape[-2:] == sample[1].shape[-2:]
        img, target = sample
        return self.resize_image(img), self.resize_target(target)


class PairwiseRandomCrop:
    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        assert isinstance(sample, tuple) and len(sample) == 2
        assert sample[0].shape[-2:] == sample[1].shape[-2:]
        img, target = sample
        # randomize cropping position
        crop_params = transforms.RandomCrop.get_params(img, self.output_size)
        img_crop = transforms.functional.crop(img, *crop_params)
        target_crop = transforms.functional.crop(target, *crop_params)
        return img_crop, target_crop

transform = transforms.Compose([
    PairwiseResize((512,1024)),
    PairwiseRandomCrop((256,256)),
    PairwiseRandomHorizontalFlip()
])

dataset = Cityscapes(root_dir, 'train', pairwise_transform=transform)
dataset.show(0)

dataloader = DataLoader(dataset, batch_size=8)

img_batch, gt_batch = iter(dataloader).next()
print(img_batch.shape)
print(gt_batch.shape)
a=1


